In [30]:
import pennylane as qml
from pennylane import numpy as np

from IPython.display import display, Latex

import scipy as sp

import matplotlib.pyplot as plt

# == Hamiltonain setup == 
Coeff_ham = [1]
Obs = [qml.PauliZ(0)]
Hamiltonian = qml.Hamiltonian(Coeff_ham, Obs)

dev = qml.device("default.mixed", wires = 1)

# == gamma -> t ==
# gamma = 0.5
# t = -np.log( np.sqrt(1 - gamma))

# == Define gammma ==
t = 1e-9
gamma_dephase = 1 - np.exp(-2 * t)

gamma_ps = 0.8

In [31]:
#  == Generate circuit ==
# Stage_1: Generate dephase
@qml.qnode(dev)
def Dephase_circuit(phi):
    qml.Hadamard(wires=0)
    
    # Time_evo t fixed as -phi/2
    qml.ApproxTimeEvolution(Hamiltonian, -phi/2, 1)
    
    qml.PhaseDamping(gamma_dephase, wires = 0)
    
    qml.Hadamard(wires=0)
    
    return qml.density_matrix(wires=0)


# Stage_2: Post-selection
@qml.qnode(dev)
def Post_selection(phi):
    rho_dephase = Dephase_circuit(phi)
    Kraus_oper = np.array([ [np.sqrt(1-gamma_ps), 0], [0, 1] ])
    
    Num = Kraus_oper @ rho_dephase @ Kraus_oper.conj().T
    Den = np.trace(Kraus_oper @ rho_dephase @ Kraus_oper.conj().T)
    rho_ps = Num / (Den)
    qml.QubitDensityMatrix(rho_ps, wires=0)
    
    return qml.density_matrix(wires=0)

In [32]:
# == CFI; cost function to maximize ==
def CFI_ps(phi):
    # Type casting for phi
    para = np.array([phi], requires_grad = 'True')

    Cost_function = qml.qinfo.classical_fisher(Post_selection)(para[0])
    # Cost_function = round( np.array(Temp, requires_grad = False).numpy(),10)
    
    # Value-bound: 0 ~ 1
    # if, 1 - cf = min -> then, cf = max
    return -Cost_function

In [33]:
print("== Status ==")
print('gamma:', gamma_ps,'  t:',t,' gamma_dephase:', gamma_dephase)

== Status ==
gamma: 0.8   t: 1e-09  gamma_dephase: 1.999999943436137e-09


In [34]:
# == Optimize begin ==

Init_phi = 0 + 1e-1 + np.pi * 2 

Seacrh_bound = np.array([0, np.pi * 4]).numpy()
Constraints = sp.optimize.Bounds([Seacrh_bound[0]], [Seacrh_bound[1]])

# SLSQP method to set constraints 
OPT_result = sp.optimize.minimize(CFI_ps, Init_phi, method = 'SLSQP', bounds = Constraints)

print(OPT_result)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -4.99936747704383
       x: [ 6.278e+00]
     nit: 6
     jac: [-1.081e-02]
    nfev: 16
    njev: 6


In [35]:
# == Check for the cost function ==
temp_bound = np.linspace(0, np.pi * 2 , 11)

data_1 = np.zeros(105)
data_2 = np.zeros(105)

Data = np.zeros((2, len(temp_bound)), requires_grad = 'False').numpy()


print('CFI_direct vs CFI_PS\n')
for i in range(len(temp_bound)):
    
    Data[0][i], Data[1][i] = qml.qinfo.classical_fisher(Post_selection)(temp_bound[i]), -CFI_ps(temp_bound[i])
    print(Data[0][i], '  ', Data[1][i])

CFI_direct vs CFI_PS

0.0    0.0
2.618033967463875    2.618033967463875
0.8812494984721811    0.8812494984721811
0.38196601053601437    0.38196601053601437
0.23445297726784584    0.23445297726784584
1.499752414512435e-24    1.499752414512435e-24
0.23445297726784609    0.23445297726784609
0.38196601053601403    0.38196601053601403
0.8812494984721808    0.8812494984721808
2.6180339674638766    2.6180339674638766
1.4997524073135885e-22    1.4997524073135885e-22


In [36]:
# == MAX CFI after optimization ==
print('== Optimization result ==\n')
print('Search bound: [', Seacrh_bound[0], ' ~ ', Seacrh_bound[1], ']')
# print('Maximum CFI =', -CFI_ps(OPT_result.x[0])[0], ' at phi = [', OPT_result.x[0], ']')

display(Latex( "Maximum CFI = " f"{-CFI_ps(OPT_result.x[0])[0]}" " at $\phi$ = " f"{OPT_result.x[0]}"))

== Optimization result ==

Search bound: [ 0.0  ~  12.566370614359172 ]


<IPython.core.display.Latex object>